# Returning values and referential transparency

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The topics discussed are:
- functions writing to the same value can produce a clash
- `k => x := x + k` clashes with `k => x := x * k`
- even worse, consider drawing a hollow square 
- we might want to define a few isolated, reusable concepts such as "draw a full line" and "draw a hollow line":

```
draw_full_line := () => s := "*****"
draw_hollow_line := () => s := "*   *"
...
```

- at this point we might want to also define the function for drawing the whole hollow square, without thinking about drawing individual lines anymore (the advantage of functions: thinking about different concepts in isolation):

```
...

draw_square := () =>
  s := ""
  draw_full_line()
  s := s + ?
  draw_hollow_line()
  s := s + ?
  draw_hollow_line()
  s := s + ?
  draw_hollow_line()
  s := s + ?
  draw_full_line()
  s := s + ?
```

- the problem we encounter here is that we want to put the result into variable `s`, but we cannot!
    - because `s` is also used by the line-drawing functions, it gets reset everytime!
- this breaks the isolation (also known as **encapsulation**) of functions
- invoking a function requires us to carefully think about how it works internally, and what variables it writes to
    - this is possible, but clearly inconvenient
    - a good programming language must adjust to logical concepts, not the other way around

## Returning
- what we are looking for is a mechanism that allows functions to produce an output without having to rely on shared global variables
    - because the variables are global, sharing them will produce some conflicts, which are considered to fall in the broad category of dangerous constructs known as _data races_ or _race conditions_
    - if each function has its own safe place to store the output, then the act of returning a result will be intrinsically isolated from all other functions
    - this increases isolation, so it is a positive property
- we introduce the new statement, `return`
- the syntax is `return E`, where `E` is any expression (also another function call!)
- the semantics simply evaluates the expression until it is a constant value:
    - $\text{eval(<return C>, S)} \rightarrow \text{<C>, S}$ when $\text{C}$ is a (constant) value
    - $\text{eval(<return E>, S)} \rightarrow \text{<return E'>, S'}$ where $\text{eval(<E>, S)} \rightarrow \text{<E'>, S'}$
- the fact that a function might result in something else other than just the `done` statement means that we also have to adjust the semantics of `call`:
    - $\text{eval(<call(return C)>, S)} \rightarrow \text{<C>, S[stack := S[stack][t]]}$ when $\text{C}$ is a (constant) value
    - $\text{eval(<call(return C; ...)>, S)} \rightarrow \text{<C>, S[stack := S[stack][t]]}$ when $\text{C}$ is a (constant) value
    - $\text{eval(<call(P)>, S)} \rightarrow \text{<call(P')>, S'}$ where $\text{eval(<P>, S)} \rightarrow \text{<P'>, S'}$
- notice that thanks to the second rule for `call`, we are ignoring the rest of the function when we return
    - this is an important property of `return`, as it "short-circuits" a function and closes it right away

## Examples of functions using `return`
- let us consider some simple examples of drawing functions that make use of the `return` statement
- each function will simply return what it was meant to draw, instead of writing it to some arbitrary variable

- the first example will be drawing a 4x4 hollow square:
    - let us begin with drawing single characters such as an asterisk or a newline: 
        - `asterisk := () => return "*"`
        - `space    := () => return " "`
        - `new_line := () => return "\n"`
    - now we can combine these results together:
        - `full_line := () => return asterisk() + asterisk() + asterisk() + asterisk() + new_line()`
        - `hollow_line := () => return asterisk() + space() + space() + asterisk() + new_line()`
    - the final result is quite elegant:
        - `hollow_square := () => return full_line() + hollow_line() + hollow_line() + full_line()`

    - [[STATE TRACE OF PROGRAM (quite long!)]]

    - notice that in this example we have not been needing variables anywhere!
    
- let us now generalize our functions so that they also accept the size of the figure as input
    - the basic functions for drawing single characters stay exactly the same
    - drawing a full line now accepts the length of the line as a parameter:
        - `full_line := n => l := ""; while n > 0: { n := n - 1; l := l + asterisk() }; return l + new_line()`
    - drawing a hollow line also accepts the length of the line as a parameter:
        - `hollow_line := n => l := ""; while n > 2: { n := n - 1; l := l + space() }; return asterisk() + l + asterisk() + new_line()`
- armed with the certainty that each of these functions works in full isolation from the rest, we can compose them together to build the final figure:
    - `hollow_square := n => l := full_line(n); while n > 2: { n := n - 1; l := l + hollow_line(n) }; return l + full_line(n)`
    - [[STATE TRACE OF PROGRAM]]
    
- we could also use the `full_line` function also to implement a (reversed) triangle, for example:
    - `triangle := n => l := ""; while n > 0: { l := l + full_line(n); n := n - 1 }; return l`
    - [[STATE TRACE OF PROGRAM]]

## Functions in Python
- we can translate the constructs we have seen so far into Python 
    - unfortunately Python offers a slightly more complex view of functions than the one we presented so far
- Python allows us to define lambda expressions and assign them to variables, but unfortunately Python has a very limited definition of lambda expressions
- Moreover, if we want to use full lambda expressions, then we need to assign them to a constant (a variable that can only be assigned once)
    - this is done with `def` and `return`
- for example, we could define a function to draw a line of asterisks as:

In [2]:
def draw_line(n):
    l = ""
    while n > 0:
        l = l + "*"
        n = n - 1
    return l
print(draw_line(5))

*****


- Notice that Python combines defining and naming the function
- Lambda expressions can also be defined in Python, but unfortunately they will directly return the only expression in their body
    - This greatly limits their usefulness, as they cannot contain multiple statements

In [3]:
f = lambda x: x + 1
print(f(10))

11


## Functions inside expressions
- Consider now the following statement: `x := f(3) + g(2)`
- It contains two function calls one after the other
- As you might recall, the first time we tackled the issue of evaluating expressions we introduced a simplifying hypothesis: that expression evaluation does not change the state
- This hypothesis must now be lifted: expression evaluation has no special status, and can change the state
- For this reason, all `eval_expr` rules that we defined so far are now "lifted" into `eval`, that is they may return a changed state instead of only the new expression for further computation
- As an example, consider the rules for evaluating a sum:
